In [3]:
%pip install cplex
%pip install docplex

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [4]:
from typing import Tuple, List
import logging

def read_hgr(file_path: str) -> Tuple[int, int, List[List[int]]]:
    """
    Read a DIMACS-like .hgr file and return the number of nodes, edges, and the edge list.

    Args:
        file_path: Path to the .hgr file

    Returns:
        Tuple containing:
        - number of nodes
        - number of edges
        - list of edges, where each edge is a list of vertex indices (1-based)

    Example format:
        c I am a comment
        p hs 6 5
        1 2
        2 3 4
        2 4 5
        1 3 6
        5 6
    """
    num_nodes = 0
    num_edges = 0
    edges = []

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()

            # Skip comments and empty lines
            if not line or line.startswith('c'):
                continue

            # Parse problem line
            if line.startswith('p'):
                parts = line.split()
                if len(parts) != 4 or parts[1] != 'hs':
                    raise ValueError(f"Invalid problem line format: {line}")
                num_nodes = int(parts[2])
                num_edges = int(parts[3])
                continue

            # Parse edge line
            vertices = [int(v) for v in line.split()]
            if not vertices:
                continue

            # Validate vertex numbers
            if any(v < 1 or v > num_nodes for v in vertices):
                raise ValueError(f"Vertex number out of range in line: {line}")

            edges.append(vertices)

    if not num_nodes or not num_edges:
        raise ValueError("Problem line not found or invalid")

    if len(edges) != num_edges:
        logging.warning(f"Expected {num_edges} edges but found {len(edges)}")

    return num_nodes, num_edges, edges


import docplex.mp.model as cplex

def solve_ilp(num_nodes, num_edges, edges, param_dict={}, print_info=False):
    mdl = cplex.Model()

    picked_vertices = mdl.binary_var_list(num_nodes, name="picked_vertices")

    for idx, edge in enumerate(edges):
        mdl.add_constraint(mdl.sum(picked_vertices[node-1] for node in edge) >= 1, "Edge_" + str(idx))

    mdl.minimize(mdl.sum(picked_vertices))

    """Set parameters on the model using a dictionary."""
    for param_path, value in param_dict.items():
        parts = param_path.split('.')
        param = mdl.parameters
        for part in parts:
            param = getattr(param, part)
        param.set(value)

    solution = mdl.solve()
    if print_info:
        mdl.print_information()
        print(mdl.parameters._params)
        print(mdl.solve_details)
        print(f"Solution size: {solution.objective_value}")
        # print(f"Sol: {solution}")
    result = {
        "solve_details": mdl.solve_details,
        "config": param_dict,
        "objective": solution.objective_value if solution else None,
        "solve_status": mdl.solve_details.status,
        "time": mdl.solve_details.time,
        "gap": mdl.solve_details.mip_relative_gap,
        "nodes": mdl.solve_details.nb_nodes_processed
    }

    return result

In [5]:
HGRPATH = "./testset/bremen_subgraph_300.hgr"

num_nodes, num_edges, edges = read_hgr(HGRPATH)
print(f"Graph has {num_nodes} nodes and {num_edges} edges")
solve_ilp(num_nodes, num_edges, edges, print_info=True)

Graph has 311 nodes and 309 edges
Model: docplex_model1
 - number of variables: 311
   - binary=311, integer=0, continuous=0
 - number of constraints: 309
   - linear=309
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
[docplex.mp.params.IntParameter(parameters.advance,1), docplex.mp.params.IntParameter(parameters.clocktype,2), docplex.mp.params.NumParameter(parameters.dettimelimit,1e+75), docplex.mp.params.IntParameter(parameters.lpmethod,0), docplex.mp.params.IntParameter(parameters.optimalitytarget,0), docplex.mp.params.IntParameter(parameters.parallel,0), docplex.mp.params.BoolParameter(parameters.paramdisplay,1), docplex.mp.params.IntParameter(parameters.qpmethod,0), docplex.mp.params.IntParameter(parameters.randomseed,202009243), docplex.mp.params.BoolParameter(parameters.record,0), docplex.mp.params.IntParameter(parameters.solutiontype,0), docplex.mp.params.IntParameter(parameters.threads,0), docplex.mp.params.NumParameter(parameters.timelimit,1e+75), doc

{'solve_details': docplex.mp.SolveDetails(time=0.532,status='integer optimal solution'),
 'config': {},
 'objective': 84.0,
 'solve_status': 'integer optimal solution',
 'time': 0.5320000000001528,
 'gap': 0.0,
 'nodes': 140}

In [6]:
from tqdm.contrib.itertools import product
# [docplex.mp.params.IntParameter(parameters.advance,1), docplex.mp.params.IntParameter(parameters.clocktype,2), docplex.mp.params.NumParameter(parameters.dettimelimit,1e+75), docplex.mp.params.IntParameter(parameters.lpmethod,0), docplex.mp.params.IntParameter(parameters.optimalitytarget,0), docplex.mp.params.IntParameter(parameters.parallel,0), docplex.mp.params.BoolParameter(parameters.paramdisplay,1), docplex.mp.params.IntParameter(parameters.qpmethod,0), docplex.mp.params.IntParameter(parameters.randomseed,202009243), docplex.mp.params.BoolParameter(parameters.record,0), docplex.mp.params.IntParameter(parameters.solutiontype,0), docplex.mp.params.IntParameter(parameters.threads,0), docplex.mp.params.NumParameter(parameters.timelimit,1e+75), docplex.mp.params.StrParameter(parameters.workdir,.), docplex.mp.params.NumParameter(parameters.workmem,2048.0)]

import pandas as pd
import numpy as np

NR_RUNS = 30
def gridsearch(param_values, runs_per_data = NR_RUNS):
	rezults = []
	param_names = list(param_values.keys())
	for cfg in product(*param_values.values()):
		# Remove None values (-> defaults)
		cfg = {param_names[i]: v for i, v in enumerate(cfg) if v is not None}
		times = [solve_ilp(num_nodes, num_edges, edges, param_dict=cfg)['time'] for _ in range(runs_per_data-1)]
		rez = solve_ilp(num_nodes, num_edges, edges, param_dict=cfg)
		times.append(rez['time'])
		rez['time_mean'] = np.mean(times)
		rez['time_std'] = np.std(times)
		rez['time_var'] = np.var(times)
		del rez['time']
		rezults.append(rez)
	return rezults

In [7]:
# 138:49:35 HOURS cu toate (cu o singura rulare)
param_values_tols_strat = {
	"mip.tolerances.mipgap":[None, 0.01, 0.05, 0.1],
	"mip.tolerances.integrality":[None, 1e-5, 1e-4, 1e-3],
	"mip.strategy.search": [None, 0, 1, 2],#auto trad dynamic
	# "lpmethod": [None, 1, 2, 3], #primal, dual, barrier
	# "mip.cuts.gomory": [None, 0, 1, 2], #off, moderate, aggressive
	# "mip.cuts.liftproj": [None, 0, 1, 2],
	# "mip.cuts.mircut": [None, 0, 1, 2],
	# "mip.cuts.zerohalfcut": [None, 0, 1, 2],
	# "mip.cuts.flowcovers": [None, 0, 1, 2],
}

# 138:49:35 HOURS cu toate (cu o singura rulare)
rezults_tols_strat = gridsearch(param_values_tols_strat)
rezults_tols_strat = pd.DataFrame(rezults_tols_strat).sort_values(['objective', 'time_mean'], ascending=[True, True])
display(rezults_tols_strat)
rezults_tols_strat.to_json("rezults_tols_strat.json")

  0%|          | 0/64 [00:00<?, ?it/s]

,solve_details,config,objective,solve_status,gap,nodes,time_mean,time_std,time_var
40,"status = integer optimal, tolerance\ntime ...","{'mip.tolerances.mipgap': 0.05, 'mip.tolerance...",84.0,"integer optimal, tolerance",0.034868,0,0.117733,0.008805,0.000078
33,"status = integer optimal, tolerance\ntime ...","{'mip.tolerances.mipgap': 0.05, 'mip.strategy....",84.0,"integer optimal, tolerance",0.034868,0,0.118767,0.010468,0.000110
39,"status = integer optimal, tolerance\ntime ...","{'mip.tolerances.mipgap': 0.05, 'mip.tolerance...",84.0,"integer optimal, tolerance",0.034868,0,0.119733,0.011647,0.000136
41,"status = integer optimal, tolerance\ntime ...","{'mip.tolerances.mipgap': 0.05, 'mip.tolerance...",84.0,"integer optimal, tolerance",0.034868,0,0.120400,0.011473,0.000132
32,"status = integer optimal, tolerance\ntime ...",{'mip.tolerances.mipgap': 0.05},84.0,"integer optimal, tolerance",0.034868,0,0.121333,0.012507,0.000156
...,...,...,...,...,...,...,...,...,...
54,"status = integer optimal, tolerance\ntime ...","{'mip.tolerances.mipgap': 0.1, 'mip.tolerances...",87.0,"integer optimal, tolerance",0.084588,0,0.039567,0.007885,0.000062
52,"status = integer optimal, tolerance\ntime ...","{'mip.tolerances.mipgap': 0.1, 'mip.tolerances...",87.0,"integer optimal, tolerance",0.084588,0,0.040600,0.010369,0.000108
63,"status = integer optimal, tolerance\ntime ...","{'mip.tolerances.mipgap': 0.1, 'mip.tolerances...",87.0,"integer optimal, tolerance",0.084588,0,0.041100,0.008669,0.000075
56,"status = integer optimal, tolerance\ntime ...","{'mip.tolerances.mipgap': 0.1, 'mip.tolerances...",87.0,"integer optimal, tolerance",0.084588,0,0.041200,0.011013,0.000121


In [ ]:
param_values_bnb = {
	"mip.strategy.search": [1],#b&b
	"lpmethod": [1, 2], #primal, dual
	"mip.cuts.gomory": [0, 1, 2], #off, moderate, aggressive
	"mip.cuts.liftproj": [0, 1, 2],
	"mip.cuts.mircut": [0, 1, 2],
	"mip.cuts.zerohalfcut": [0, 1, 2],
}

rezults_bnb = gridsearch(param_values_bnb)
rezults_bnb = pd.DataFrame(rezults_bnb).sort_values(['objective', 'time_mean'], ascending=[True, True])
display(rezults_bnb)
rezults_bnb.to_json("rezults_bnb.json")

  0%|          | 0/162 [00:00<?, ?it/s]

In [ ]:
param_values_th = {
	"threads": [None, 1],
}

rezults_th = gridsearch(param_values_th)
rezults_th = pd.DataFrame(rezults_th).sort_values(['objective', 'time_mean'], ascending=[True, True])
display(rezults_th)